In [6]:
import pandas as pd
from tqdm import tqdm
from tmdbv3api import Collection

import sys
sys.path.insert(0, '../data_prep')
from sqlite_utils import select_statement_to_df, get_from_table

In [7]:
all_collections_statement = """

SELECT COLLECTION_ID, COLLECTION_NAME, COUNT(*) AS FILM_COUNT
FROM FILM_COLLECTIONS
WHERE COLLECTION_ID != -1
GROUP BY COLLECTION_ID

"""

In [8]:
all_collections_df = select_statement_to_df(all_collections_statement)
all_collections_df

,COLLECTION_ID,COLLECTION_NAME,FILM_COUNT
0,10,Star Wars Collection,9
1,84,Indiana Jones Collection,5
2,119,The Lord of the Rings Collection,3
3,131,Three Colors Collection,3
4,151,Star Trek: The Original Series Collection,6
...,...,...,...
886,1120657,Cliffhanger Collection,1
887,1125292,Freaky Friday Collection,1
888,1126392,El hoyo - Colección,1
889,1129084,Nobody Collection,1


In [9]:
def check_if_tmdb_id_ingested(tmdb_id):
    film_ingested = 0
    if len(select_statement_to_df('SELECT FILM_ID FROM TMDB_ID WHERE TMDB_ID = {}'.format(tmdb_id)).values) == 1:
        film_ingested = 1
    return film_ingested

In [10]:
def get_missing_films(collection_id):
    collection = Collection()
    collection_details = collection.details(collection_id)
    tmp_df = pd.DataFrame([{'TMDB_ID': x.id, 'FILM_TITLE': x.title, 'FILM_YEAR': x['release_date'][:4], 'COLLECTION_ID': collection_id} for x in collection_details['parts']])
    tmp_df['MISSING'] = 1 - tmp_df['TMDB_ID'].apply(check_if_tmdb_id_ingested)
    return tmp_df

In [11]:
all_collection_films = []
bad_collection_ids = []
for collection_id in tqdm(all_collections_df['COLLECTION_ID']):
    try:
        all_collection_films.append(get_missing_films(collection_id))
    except:
        bad_collection_ids.append(collection_id)

100%|██████████| 891/891 [00:54<00:00, 16.23it/s]


In [12]:
bad_collection_ids

[578704, 768464]

In [13]:
all_collection_films_df = pd.concat(all_collection_films).reset_index(drop=True)
all_collection_films_df

,TMDB_ID,FILM_TITLE,FILM_YEAR,COLLECTION_ID,MISSING
0,11,Star Wars,1977,10,0
1,1891,The Empire Strikes Back,1980,10,0
2,1892,Return of the Jedi,1983,10,0
3,1893,Star Wars: Episode I - The Phantom Menace,1999,10,0
4,1894,Star Wars: Episode II - Attack of the Clones,2002,10,0
...,...,...,...,...,...
2862,615457,Nobody,2021,1129084,0
2863,1007734,Nobody 2,,1129084,1
2864,18311,Days of Being Wild,1990,1131062,1
2865,843,In the Mood for Love,2000,1131062,0


In [14]:
MISSING = all_collection_films_df[all_collection_films_df['MISSING'] == 1].reset_index(drop=True)

In [15]:
len(MISSING)

722

In [16]:
round = -1
increment = 50

In [17]:
round += 1
MISSING.head(increment*(round+1)).tail(increment)

,TMDB_ID,FILM_TITLE,FILM_YEAR,COLLECTION_ID,MISSING
0,487560,Last Friday,,43563,1
1,983058,National Treasure 3,,52984,1
2,9255,Hot Shots! Part Deux,1993,52988,1
3,12107,Nutty Professor II: The Klumps,2000,86028,1
4,47743,The Saint in New York,1938,86224,1
5,45571,The Saint's Double Trouble,1940,86224,1
6,31723,Women in Trouble,2009,86668,1
7,893060,The Boondock Saints III,,87186,1
8,12310,The Four Musketeers,1974,87631,1
9,269650,Appleseed Alpha,2014,87800,1
